In [ ]:
#This is MLR with 15 sets of NLM genes

In [ ]:
import sys
sys.executable

In [ ]:
#!'/Library/Developer/CommandLineTools/usr/bin/python3' -m pip install smote_variants

In [ ]:
#!'/Library/Developer/CommandLineTools/usr/bin/python3' -m pip install imblearn

In [1]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from numpy import mean
from numpy import std
from sklearn import metrics
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

In [2]:
import smote_variants as sv

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
pd.options.mode.chained_assignment = None

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
import seaborn as sns
#import matplotlib.pyplot as plt


2022-05-08 07:49:30,242:DEBUG:matplotlib data path: /Users/madhuri/Library/Python/3.8/lib/python/site-packages/matplotlib/mpl-data
2022-05-08 07:49:30,244:DEBUG:CONFIGDIR=/Users/madhuri/.matplotlib
2022-05-08 07:49:30,245:DEBUG:interactive is False
2022-05-08 07:49:30,246:DEBUG:platform is darwin
2022-05-08 07:49:30,246:DEBUG:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_frozen_importlib_external', '_io', 'marshal', 'posix', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings.cp437', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', '_distutils_hack', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator', 'keyword', '_heapq', 'heapq', 'iter

2022-05-08 07:49:30,301:DEBUG:CACHEDIR=/Users/madhuri/.matplotlib
2022-05-08 07:49:30,302:DEBUG:Using fontManager instance from /Users/madhuri/.matplotlib/fontlist-v330.json
2022-05-08 07:49:30,536:DEBUG:Loaded backend module://matplotlib_inline.backend_inline version unknown.
2022-05-08 07:49:30,538:DEBUG:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [6]:
clinical = pd.read_csv('/Users/madhuri/Desktop/MasterThesis/Part2/ClinicalDF1.csv')

In [7]:
#gene_data = pd.read_csv('/Users/madhuri/Desktop/MasterThesis/Part2/final_gene_list.csv')

In [8]:
import os
file_list=[]
#path_of_the_directory = '/storage/home/mpg5798/work/Non LandmarkGenes'
for files in os.listdir():
    if ((files.startswith("NLM")) & (files.endswith('.csv'))):
        file_list.append(files)
print(file_list)

['NLM_Genes8.csv', 'NLM_Genes9.csv', 'NLM_Genes7.csv', 'NLM_Genes12.csv', 'NLM_Genes13.csv', 'NLM_Genes6.csv', 'NLM_Genes4.csv', 'NLM_Genes11.csv', 'NLM_Genes10.csv', 'NLM_Genes5.csv', 'NLM_Genes1.csv', 'NLM_Genes14.csv', 'NLM_Genes0.csv', 'NLM_Genes2.csv', 'NLM_Genes3.csv']


In [10]:
r_state=[20,30,40,50,60,70,80,90]
for r in r_state:
    print("(((((((((((())))))))))))", r)
    accuracies=[]
    for i in range(15):
        gene_data = pd.read_csv(file_list[i])
        dataframe=gene_data.iloc[:-1,1:]
    #     print(dataframe.head())
    #     print(dataframe.shape)
        dataframe['CancerType']=clinical['CancerType']
        X= dataframe.iloc[:, :-1]
        y=dataframe['CancerType']
        X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=r) 
        le = LabelEncoder()
        le.fit(y_train)
        y_train_enc= le.transform(y_train)
        y_test_enc=le.transform(y_test)
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_scaled = scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        # Define SMOTE-Tomek Links
        resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
        X_samp, y_samp = resample.fit_resample(X_scaled, y_train_enc)
        #define the multinomial logistic regression model
        model = LogisticRegression(multi_class='multinomial', solver='sag', max_iter = 2000)
        # define the model evaluation procedure
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)#make n_repeats=10
        # evaluate the model and collect the scores
        n_scores = cross_val_score(model, X_samp, y_samp, scoring='accuracy', cv=cv, n_jobs=-1)
        # report the model performance
        print("Training Accuracy")
        print("Results for set:", file_list[i])
        print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores))) #Training Accuracy
        model.fit(X_samp,y_samp)
        y_pred=model.predict(X_test)
        #print("(((((((((((())))))))))))", r)
        #print("Results for set:", file_list[i])
        #print("f1 score:",f1_score(y_test_enc, y_pred, average='macro'))
        acc=round((metrics.accuracy_score(y_test_enc,y_pred)*100),3)

        accuracies.append(acc)
        #print(metrics.classification_report(y_test_enc, y_pred, digits=3))

    print(accuracies)
    with open('nlm_15_acc.txt', 'a') as output_file:
        for acc in accuracies:
            output_file.write(str(acc) + '\n')
    print("#####################################################")

(((((((((((()))))))))))) 20
Results for set: NLM_Genes8.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes9.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes7.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes12.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes13.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes6.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes4.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes11.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes10.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes5.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes1.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes14.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes0.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes2.csv
(((((((((((()))))))))))) 20
Results for set: NLM_Genes3.csv
[86.281, 85.791, 86.379, 85.742, 86.722, 85.889, 85.497, 86.82, 85.791, 85.693, 85.007, 86.673,

(((((((((((()))))))))))) 90
Results for set: NLM_Genes14.csv
(((((((((((()))))))))))) 90
Results for set: NLM_Genes0.csv
(((((((((((()))))))))))) 90
Results for set: NLM_Genes2.csv
(((((((((((()))))))))))) 90
Results for set: NLM_Genes3.csv
[86.33, 85.644, 88.045, 86.232, 86.477, 85.889, 86.477, 87.31, 87.016, 86.526, 85.007, 88.192, 86.722, 86.232, 86.232]
#####################################################


In [39]:
round(90.12345788990889,2)

90.12

#max_iter=1500 [85.25232729054385, 85.15433610975012, 87.55512003919648, 85.35031847133759, 87.26114649681529, 86.13424791768742, 84.66438020578147, 86.33023027927487, 86.72219500244978, 86.13424791768742, 84.90935815776581, 85.88926996570308, 85.20333170014699, 86.67319941205291, 85.10534051935326]

In [ ]:
with open('nlm_15_acc.txt', 'w') as output_file:
    for acc in accuracies:
        output_file.write(acc + '\n')